In [1]:
# NHSRC PHC SUPPLY CHAIN - REPLENISHMENT POLICY ENGINE
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

print("🏥 NHSRC PHC REPLENISHMENT POLICY ENGINE")
print("=" * 70)

# 1️⃣ LOAD INVENTORY & BEST-MODEL FORECASTS
print("📥 1. Loading Data Sources...")

# Load forecast-ready time series
df = pd.read_csv("data/forecast_ready_timeseries.csv")
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')

# Load best model selections
best_models = pd.read_csv("reports/best_model_selection.csv")

# Load stock health matrix
stock_health = pd.read_csv("reports/stock_health_matrix.csv")

print(f"   Time series data: {len(df):,} records, {df['sku_id'].nunique()} SKUs")
print(f"   Best models: {len(best_models):,} selections")
print(f"   Stock health: {len(stock_health):,} SKU assessments")

# 2️⃣ FORECAST 14 DAYS FORWARD PER SKU
print("\n🔮 2. Generating 14-Day Forecasts Using Best Models...")

def forecast_next_14_days(series, method):
    """Generate 14-day forecast based on selected model type"""
    if len(series) < 30:
        return series.mean()  # Fallback for insufficient data
    
    try:
        if method == "naive":
            # Naive: Use last observed value as daily forecast
            daily_forecast = series.iloc[-1]
            return daily_forecast * 14  # 14-day total
        
        elif method.startswith("ma_"):
            # Moving Average: window from method name (ma_7, ma_14, ma_30)
            window = int(method.split("_")[1])
            if len(series) >= window:
                daily_forecast = series.iloc[-window:].mean()
            else:
                daily_forecast = series.mean()
            return daily_forecast * 14  # 14-day total
        
        elif method == "ets":
            # Exponential Smoothing (simple implementation)
            # Simple weighted average with more weight on recent observations
            weights = np.exp(np.linspace(-1, 0, min(30, len(series))))
            weights /= weights.sum()
            recent_data = series.iloc[-len(weights):].values
            daily_forecast = np.dot(recent_data, weights)
            return daily_forecast * 14  # 14-day total
        
        else:
            # Default fallback: 7-day moving average
            daily_forecast = series.iloc[-7:].mean() if len(series) >= 7 else series.mean()
            return daily_forecast * 14
            
    except Exception as e:
        print(f"   ⚠️  Forecast error for method {method}: {e}")
        daily_forecast = series.mean()
        return daily_forecast * 14

forecast_output = []

print("   Generating forecasts SKU-by-SKU:")
for i, row in best_models.iterrows():
    sku = row["sku_id"]
    method = row["model_type"]
    sku_name = row["sku_name"]
    
    # Get SKU time series
    sku_series = df[df["sku_id"] == sku]["units_used"]
    
    if len(sku_series) < 7:
        print(f"   ⚠️  {sku}: Insufficient data ({len(sku_series)} records)")
        forecast_14d = sku_series.mean() * 14 if len(sku_series) > 0 else 0
    else:
        forecast_14d = forecast_next_14_days(sku_series, method)
    
    forecast_output.append([sku, sku_name, method, forecast_14d])
    print(f"   {sku}: {method:6} → {forecast_14d:6.1f} units (14-day forecast)")

forecast_df = pd.DataFrame(
    forecast_output, 
    columns=["sku_id", "sku_name", "model_type", "forecast_14d"]
)

print(f"\n✅ Generated forecasts for {len(forecast_df)} SKUs")

# 3️⃣ MERGE INVENTORY HEALTH LAYER
print("\n🔗 3. Merging Forecasts with Inventory Health Data...")

# Merge stock health with forecasts
merged = pd.merge(
    stock_health, 
    forecast_df[["sku_id", "model_type", "forecast_14d"]], 
    on="sku_id", 
    how="left"
)

# Also merge VED/FSN from time series data for consistency
sku_metadata = df[["sku_id", "ved_category", "fsn_category"]].drop_duplicates()
merged = pd.merge(merged, sku_metadata, on="sku_id", how="left", suffixes=('', '_meta'))

# Use metadata if original columns missing
if 'ved_category' not in merged.columns and 'ved_category_meta' in merged.columns:
    merged['ved_category'] = merged['ved_category_meta']
if 'fsn_category' not in merged.columns and 'fsn_category_meta' in merged.columns:
    merged['fsn_category'] = merged['fsn_category_meta']

print(f"   Merged dataset: {len(merged):,} records")
print(f"   Columns: {len(merged.columns)}")

# 4️⃣ COMPUTE OPERATIONAL DECISION LOGIC
print("\n🧮 4. Computing Operational Decision Logic...")

# Calculate expected days available
merged["expected_days_available"] = merged.apply(
    lambda x: x["current_stock"] / (x["forecast_14d"] / 14) if x["forecast_14d"] > 0 else 999,
    axis=1
)

# Reorder flag based on lead time
merged["reorder_flag"] = merged.apply(
    lambda x: "YES" if x["expected_days_available"] < x["lead_time_days"] else "NO",
    axis=1
)

# Recommended order quantity (capped at reasonable limits)
merged["recommended_order_qty"] = merged.apply(
    lambda x: max(x["ROL"] - x["current_stock"], 0) if x["reorder_flag"] == "YES" else 0,
    axis=1
)

# Calculate safety stock coverage
merged["safety_stock_coverage"] = merged.apply(
    lambda x: x["current_stock"] / x["safety_stock"] if x["safety_stock"] > 0 else 999,
    axis=1
)

print("   Computed metrics:")
print(f"   - Expected days available: {merged['expected_days_available'].min():.1f} to {merged['expected_days_available'].max():.1f} days")
print(f"   - SKUs needing reorder: {(merged['reorder_flag'] == 'YES').sum()} of {len(merged)}")
print(f"   - Total recommended order quantity: {merged['recommended_order_qty'].sum():,.0f} units")

# 5️⃣ APPLY NHSRC PRIORITY TIERING LOGIC
print("\n🏥 5. Applying NHSRC Priority Tiering Logic...")

def classify_procurement_action(row):
    """NHSRC-compliant procurement classification"""
    
    # URGENT: Vital items with immediate stockout risk
    if (row["ved_category"] == "Vital" and 
        row["expected_days_available"] < 7):
        return "URGENT REPLENISH"
    
    # CRITICAL: Any item with stockout risk within lead time
    if row["reorder_flag"] == "YES":
        if row["ved_category"] == "Vital":
            return "PRIORITY 1: REORDER NOW"
        elif row["ved_category"] == "Essential":
            return "PRIORITY 2: REORDER SOON"
        else:
            return "PRIORITY 3: PLAN REORDER"
    
    # EXPIRY RISK: Items with imminent expiry
    # Note: expiry_risk_bucket not in current data, using days_cover as proxy
    if "expiry_days_remaining" in row and row["expiry_days_remaining"] < 30:
        return "REDISTRIBUTE / USE FIRST"
    elif row["days_cover"] > 90:  # Excess stock indicator
        return "MONITOR: EXCESS STOCK"
    
    # HEALTHY: Adequate stock with good coverage
    if (row["expected_days_available"] > row["lead_time_days"] * 1.5 and
        row["safety_stock_coverage"] > 1.5):
        return "HOLD: HEALTHY STOCK"
    
    # DEFAULT: Normal monitoring
    return "MONITOR: NORMAL"

merged["procurement_action"] = merged.apply(classify_procurement_action, axis=1)

# Add action priority score (1=Highest, 5=Lowest)
def assign_priority(action):
    priority_map = {
        "URGENT REPLENISH": 1,
        "PRIORITY 1: REORDER NOW": 1,
        "PRIORITY 2: REORDER SOON": 2,
        "REDISTRIBUTE / USE FIRST": 2,
        "PRIORITY 3: PLAN REORDER": 3,
        "MONITOR: EXCESS STOCK": 4,
        "HOLD: HEALTHY STOCK": 5,
        "MONITOR: NORMAL": 4
    }
    return priority_map.get(action, 4)

merged["action_priority"] = merged["procurement_action"].apply(assign_priority)

# 6️⃣ ADD FORECAST CONFIDENCE METRICS
print("\n📊 6. Adding Forecast Confidence Metrics...")

# Calculate forecast variability score
merged["forecast_variability"] = merged.apply(
    lambda x: x["ADC_std"] / x["ADC"] if x["ADC"] > 0 else 1.0,
    axis=1
)

# Assign confidence levels
def assign_confidence(variability):
    if variability < 0.3:
        return "HIGH"
    elif variability < 0.6:
        return "MEDIUM"
    else:
        return "LOW"

merged["forecast_confidence"] = merged["forecast_variability"].apply(assign_confidence)

print("   Forecast confidence distribution:")
confidence_counts = merged["forecast_confidence"].value_counts()
for conf, count in confidence_counts.items():
    print(f"   - {conf}: {count} SKUs")

# 7️⃣ SAVE OUTPUT
print("\n💾 7. Saving Replenishment Recommendations...")

# Select and order final columns
final_columns = [
    "sku_id", "sku_name", "ved_category", "fsn_category",
    "current_stock", "ADC", "ADC_std", "lead_time_days",
    "safety_stock", "ROL", "MSL",
    "model_type", "forecast_14d", "expected_days_available",
    "days_cover", "reorder_flag", "recommended_order_qty",
    "procurement_action", "action_priority",
    "forecast_confidence", "forecast_variability"
]

# Keep only columns that exist
available_columns = [col for col in final_columns if col in merged.columns]
final_df = merged[available_columns].sort_values("action_priority")

# Save to CSV
output_path = "reports/replenishment_recommendations.csv"
final_df.to_csv(output_path, index=False)
print(f"   ✅ Saved: {output_path}")
print(f"   Records: {len(final_df)}")
print(f"   Columns: {len(final_df.columns)}")

# 8️⃣ ANALYSIS SUMMARY
print("\n📈 8. Replenishment Analysis Summary:")

# Action distribution
action_distribution = final_df["procurement_action"].value_counts()
print("\n   PROCUREMENT ACTION DISTRIBUTION:")
for action, count in action_distribution.items():
    percentage = (count / len(final_df)) * 100
    print(f"   - {action}: {count} SKUs ({percentage:.1f}%)")

# Financial impact estimate
financial_impact = final_df["recommended_order_qty"].sum() * final_df["price_per_unit"].mean() if "price_per_unit" in final_df.columns else 0
print(f"\n   ESTIMATED FINANCIAL IMPACT:")
print(f"   - Total units to order: {final_df['recommended_order_qty'].sum():,.0f}")
if financial_impact > 0:
    print(f"   - Estimated cost: ₹{financial_impact:,.0f}")

# Stockout risk summary
stockout_risk = final_df[final_df["expected_days_available"] < 7]
print(f"\n   STOCKOUT RISK ASSESSMENT:")
print(f"   - Critical risk (<7 days): {len(stockout_risk)} SKUs")
if len(stockout_risk) > 0:
    print(f"   - Includes: {', '.join(stockout_risk['sku_name'].head(3).tolist())}")

# 9️⃣ FINAL OUTPUTS FOR TRAINER
print("\n" + "="*70)
print("🎯 TRAINER OUTPUTS")
print("="*70)

print("\n1. 🔹 HEAD OF REPLENISHMENT_RECOMMENDATIONS.CSV (10 ROWS):")
print("-" * 70)
print(final_df.head(10).to_string())

print("\n2. 🔹 SKU COUNTS BY PROCUREMENT_ACTION:")
print("-" * 70)
for action, count in action_distribution.items():
    print(f"   {action}: {count} SKUs")

print("\n3. 🔹 UPDATED GIT LS-FILES:")
print("-" * 70)
import subprocess
result = subprocess.run(['git', 'ls-files'], capture_output=True, text=True)
print(result.stdout)

print("\n" + "="*70)
print("✅ DAY 6 REPLENISHMENT POLICY ENGINE COMPLETE")
print("="*70)
print("\n📌 Key Business Outcomes:")
print("   • Automated procurement decisions for all 12 SKUs")
print("   • NHSRC-compliant priority tiering implemented")
print("   • Stockout risk quantified and actionable")
print("   • Ready for warehouse and procurement team execution")

🏥 NHSRC PHC REPLENISHMENT POLICY ENGINE
📥 1. Loading Data Sources...
   Time series data: 2,160 records, 12 SKUs
   Best models: 12 selections
   Stock health: 12 SKU assessments

🔮 2. Generating 14-Day Forecasts Using Best Models...
   Generating forecasts SKU-by-SKU:
   MED001: ma_30  → 1550.3 units (14-day forecast)
   MED002: ma_7   → 1318.0 units (14-day forecast)
   MED003: ma_30  → 1062.6 units (14-day forecast)
   MED004: ets    →  999.9 units (14-day forecast)
   MED005: ma_7   →  724.0 units (14-day forecast)
   MED006: ets    →  122.2 units (14-day forecast)
   MED007: naive  →   56.0 units (14-day forecast)
   MED008: ets    →  232.8 units (14-day forecast)
   MED009: ets    →  371.5 units (14-day forecast)
   MED010: ets    →  179.9 units (14-day forecast)
   MED011: ets    →   76.9 units (14-day forecast)
   MED012: ma_7   →  270.0 units (14-day forecast)

✅ Generated forecasts for 12 SKUs

🔗 3. Merging Forecasts with Inventory Health Data...
   Merged dataset: 12 records